deo

In [1]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,Lambda
from keras.utils import to_categorical
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from keras import backend as K
import gensim

In [2]:
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance."""
data

'Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.'

In [3]:
sentences = data.split('.')
data=sentences
data

['Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning',
 ' Learning can be supervised, semi-supervised or unsupervised',
 ' Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance',
 '']

In [4]:
vectorize=Tokenizer()#divides the data into tokens
data = [str(item) for item in data] #not mandattory but converts no strigns to strigns
vectorize.fit_on_texts(data)#gives each unique word a diff number mostly interger only maps it 
data=vectorize.texts_to_sequences(data)#based on the above numbering the words are converted to numbers like eah sentecne is a list of numbers
total_vocab=sum(len(s) for s in data)#will count the total words in the dataset matlab summing lens of each vector/sentecne
word_count=len(vectorize.word_index)+1#will count the total unique words +1 is for padding sequences to the same length common in neural networks
window_size=2

In [5]:
def cbow_model(data,window_size,total_vocab):
    total_length=window_size*2
    for text in data:
        text_len=len(text)
        for idx,word in enumerate(text):
            context_word=[]
            #text_len=len(text)
            target=[]
            begin=idx-window_size
            end=idx+window_size+1
            context_word.append([text[i] for i in range(begin,end) if 0<=i<text_len and i!=idx])
            target.append(word)
            contextual=sequence.pad_sequences(context_word,maxlen=total_length)
            final_target=to_categorical(target,total_vocab)
            yield(contextual,final_target)
        
        


In [6]:
model=Sequential()
model.add(Embedding(input_dim=total_vocab,output_dim=100,input_length=window_size*2))
model.add(Lambda(lambda x: keras.backend.mean(x,axis=1),output_shape=(100,)))
model.add(Dense(total_vocab,activation='softmax'))


C:\Users\smara\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [7]:
model.compile(loss='categorical_crossentropy',optimizer='adam')


In [8]:
for i in range(5):
    cost=0
    for x,y in cbow_model(data,window_size,total_vocab):
        cost+=model.train_on_batch(x,y)
        print(i,cost)

0 4.631167888641357
0 9.259047985076904
0 13.883515357971191
0 18.50191307067871
0 23.12058401107788
0 27.735761642456055
0 32.347922801971436
0 36.956913471221924
0 41.56343698501587
0 46.168363094329834
0 50.77353048324585
0 55.37958240509033
0 59.98583984375
0 64.59259462356567
0 69.19675207138062
0 73.80039405822754
0 78.40173530578613
0 83.00251865386963
0 87.6036286354065
0 92.20476198196411
0 96.80748176574707
0 101.40985345840454
0 106.01221656799316
0 110.614182472229
0 115.2177300453186
0 119.81999444961548
0 124.4216775894165
0 129.0235447883606
0 133.626690864563
0 138.22987127304077
0 142.83342266082764
0 147.43676614761353
0 152.04021787643433
0 156.64396858215332
0 161.24519157409668
0 165.84500360488892
0 170.44523191452026
0 175.04578495025635
0 179.6456561088562
0 184.2439308166504
0 188.84174394607544
0 193.4397473335266
0 198.03623294830322
0 202.63312435150146
0 207.2304949760437
0 211.82626008987427
0 216.42187213897705
0 221.01613092422485
0 225.61105108261108
0 

In [13]:
import gensim

# Define dimensions and absolute file path
dimensions = 100  # Make sure this matches the actual dimensions of your word vectors
file_path = r'C:\Users\smara\Desktop\SPPU-SE-TE-BE-Assignments-main\Semester 7\timepass\vectors.txt'

# Get the weights from your model
weights = model.get_weights()[0]  # Assuming this is a 2D array with shape (vocab_size, dimensions)

# Write the vectors to the file
with open(file_path, 'w') as vect_file:
    total_vocab = len(vectorize.word_index)
    vect_file.write(f"{total_vocab} {dimensions}\n")  # Write header with vocab size and dimensions
    
    for word, index in vectorize.word_index.items():
        if index < len(weights):  # Ensure index is within bounds
            # Convert weights to string and join with spaces
            vector_string = ' '.join(map(str, weights[index]))
            vect_file.write(f"{word} {vector_string}\n")

# Load the vectors with Gensim
try:
    cbow_output = gensim.models.KeyedVectors.load_word2vec_format(file_path, binary=False)
    similar_words = cbow_output.most_similar(positive=['deep'])
    print("Similar words to 'deep':", similar_words)
except FileNotFoundError as e:
    print("File not found:", e)
except gensim.models.KeyedVectors.KeyedVectorsException as e:
    print("Gensim loading error:", e)
except Exception as e:
    print("An error occurred while loading the vectors:", e)


Similar words to 'deep': [('recurrent', 0.7197873592376709), ('known', 0.6701033115386963), ('representation', 0.6655187010765076), ('neural', 0.6510266661643982), ('learning', 0.6459328532218933), ('reinforcement', 0.6349892616271973), ('such', 0.6283373832702637), ('architectures', 0.6166208386421204), ('as', 0.6104650497436523), ('also', 0.6023434996604919)]
